### GPU info ###

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

### Setup ###


In [1]:
import os
import sys
from google.colab import drive, files
from google.colab.patches import cv2_imshow

SRC_PATH = '/content/gdrive/MyDrive/TFG/src'

# mount Google Drive
if not os.path.isdir('/content/gdrive'):
  drive.mount('/content/gdrive')

# add src to sys.path
if SRC_PATH not in sys.path:
  sys.path.append(SRC_PATH)

# initialize the basic settings and import all the dependencies
import common

common.init()

from SAISOD import *

Mounted at /content/gdrive


#### Settings ####

In [2]:
# Base detection (x1)
dict_base = {
    'initial_det': True,
    'apply_sr': False,
    'apply_flip': False,
    'apply_lut': False,
    'iou_threshold': 0.25,
    'threshold_low': 0.3,
    'det_size': 1024,
    'det_model_name': 'efficientdet_d4_coco17_tpu-32',
    'det_model_date': '20200711'
}

# Base detection (x1) and SR (x2)
dict_sr_method = {
    'initial_det': True,
    'apply_sr': True,
    'apply_flip': False,
    'apply_lut': False,
    'tiling_factor': 2,
    'iou_threshold': 0.25,
    'threshold_low': 0.3,
    'det_size': 1024,
    'det_model_name': 'efficientdet_d4_coco17_tpu-32',
    'det_model_date': '20200711'
}

# Base detection (x1), SR (x2), LUT and FLIP
dict_final_method = {
    'initial_det': True,
    'apply_sr': True,
    'apply_flip': True,
    'apply_lut': True,
    'lut_threshold': 50,
    'lut_method': 'clahe',
    'tiling_factor': 2,
    'iou_threshold': 0.25,
    'threshold_low': 0.3,
    'det_size': 1024,
    'det_model_name': 'efficientdet_d4_coco17_tpu-32',
    'det_model_date': '20200711'
}

#common.load_settings(dict_final_method)
#common.print_settings()

### Detection model ###

In [ ]:
detect_fn = get_detection_function(common.DET_MODEL_NAME, common.DET_MODEL_DATE)

### Demo ###

##### declare demo images #####

In [ ]:
# test images
image_list = [
    '9999993_00000_d_0000018.jpg'
]

# figure images
"""
image_list = [
    '9999938_00000_d_0000179.jpg',
    '0000321_00000_d_0000015.jpg',
    '9999979_00000_d_0000045.jpg'
]
"""

# demo images
"""
image_list = [
  '0000006_00611_d_0000002.jpg',
  '0000272_01500_d_0000004.jpg',
  '9999938_00000_d_0000216.jpg',
  '9999938_00000_d_0000376.jpg',
  '9999979_00000_d_0000034.jpg',
  '9999996_00000_d_0000025.jpg'
]
"""
# demo images with low brightness
dark_image_list = [
    '9999952_00000_d_0000145.jpg',
    '9999952_00000_d_0000143.jpg',
    '9999952_00000_d_0000144.jpg'
]

##### run demo tests #####

In [26]:

base_det_dict = run_test(dict_base, detect_fn, image_list)
sr_det_dict = run_test(dict_sr_method, detect_fn, image_list)
prop_det_dict = run_test(dict_final_method, detect_fn, image_list)
gt_dict = get_dict_from_gt('test_gt', image_list, common.CATEGORIES_WHITELIST)

##### show demo results #####

In [ ]:
from figurehelper import *

image_id = 1
image_file = image_list[image_id - 1]

base_ann_list = get_annotations_by_image_id(image_id, base_det_dict)
sr_ann_list = get_annotations_by_image_id(image_id, sr_det_dict)
prop_ann_list = get_annotations_by_image_id(image_id, prop_det_dict)
gt_ann_list = get_annotations_by_image_id(image_id, gt_dict)

show_test_results(image_file, base_ann_list, gt_ann_list, False, True)

##### compare two method results on image #####

In [ ]:
%matplotlib inline

compare_two_results(image_file, base_ann_list, prop_ann_list)

### Run and evaluate tests ###

In [ ]:
image_list = get_image_list_from_json_file(common.UTILS_DIR, 'vd_img_testset', 300)

In [ ]:
test_dict = run_test(dict_sr_method, detect_fn, image_list)

In [ ]:
write_annotations_to_json(common.DETEVAL_DIR, 'test300_d4_0_2_sr_iou_025', test_dict)

In [ ]:
evaluate_test('test300_d4_0_2_sr_iou_025.json', 'visdrone_gt.json')

### Detection model evaluation ###

In [ ]:
evaluate_det_model(MODEL_NAME, MODEL_DATE, 0.2, None, CATEGORIES_WHITELIST)

### Super-Resolution model evaluation ###

In [ ]:
original_list = get_all_files_by_ext(os.path.join(SREVAL_DIR, 'original'),'jpg')
results = evaluate_sr_model(original_list, 'BSRGAN', 4)
print(results)


#### SR images from (EDSR, ESPCN, FSRCNN, LapSRN) ####

In [ ]:
lr_list = get_all_files_by_ext(os.path.join(SREVAL_DIR,'LR_x4'), 'jpg')
print(lr_list)
generate_sr_images(lr_list, 'FSRCNN', 2)

##### DBPI #####

In [ ]:
%cd /content

!git clone https://github.com/prote376/DBPI-BlindSR.git

/content
Cloning into 'DBPI-BlindSR'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 42 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (42/42), done.


In [ ]:
%cd /content/DBPI-BlindSR/

#!python train.py -i /content/gdrive/MyDrive/TFG/sreval/LR_x2/ -o /content/gdrive/MyDrive/TFG/sreval/test/
!python train.py -i /content/gdrive/MyDrive/TFG/sreval/test/ -o /content/gdrive/MyDrive/TFG/sreval/test/ --X4

/content/DBPI-BlindSR
Scale Factor: X4
************************************************************
STARTED DBPI-BlindSR on: "/content/gdrive/MyDrive/TFG/sreval/test/original.jpg"...
100%|███████████████████| 3000/3000 [00:34<00:00, 86.46it/s]
FINISHED RUN (see --/content/gdrive/MyDrive/TFG/sreval/test/original-- folder)
************************************************************




#### FeMaSR ####

In [ ]:
# git clone this repository
%cd /content
!pip install einops
!git clone https://github.com/chaofengc/FeMaSR.git
%cd /content/FeMaSR

# create new anaconda env
!conda create -n femasr python=3.8
!source activate femasr

# install python dependencies
!pip3 install -r requirements.txt
!python setup.py develop

In [ ]:
%cd /content/FeMaSR

!python inference_femasr.py -s 4 -i /content/gdrive/MyDrive/TFG/sreval/LR_x4 -o /content/gdrive/MyDrive/TFG/sreval/FeMaSR_x4

/content/FeMaSR
Downloading: "https://github.com/chaofengc/FeMaSR/releases/download/v0.1-pretrain_models/FeMaSR_SRX4_model_g.pth" to /root/.cache/torch/hub/checkpoints/FeMaSR_SRX4_model_g.pth

100% 159M/159M [00:01<00:00, 93.7MB/s]
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Test 9999993_00000_d_0000018.jpg: 100% 12/12 [00:17<00:00,  1.46s/image]


#### Real-ESRGAN ####

In [ ]:
!python inference_realesrgan.py -n RealESRGAN_x2plus -i /content/gdrive/MyDrive/TFG/subimages/9999952_00000_d_0000024/ -o /content/gdrive/MyDrive/TFG/subimages/9999952_00000_d_0000024/ -s 2
#!python inference_realesrgan.py -n RealESRGAN_x4plus -i /content/gdrive/MyDrive/TFG/sreval/test/ -o /content/gdrive/MyDrive/TFG/sreval/test/ -s 4

#### BSRGAN ####

In [ ]:
%cd /content/
!git clone https://github.com/cszn/BSRGAN.git

/content
Cloning into 'BSRGAN'...
remote: Enumerating objects: 541, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 541 (delta 144), reused 105 (delta 101), pack-reused 386
Receiving objects: 100% (541/541), 168.90 MiB | 46.83 MiB/s, done.
Resolving deltas: 100% (196/196), done.


In [ ]:
%cd /content/BSRGAN/

#!cp -r /content/gdrive/MyDrive/TFG/sreval/LR_x4/*.jpg /content/BSRGAN/testsets/RealSRSet
#!python main_download_pretrained_models.py
!python main_test_bsrgan.py

/content/BSRGAN
LogHandlers setup!
22-09-20 18:53:40.485 :       Model Name : BSRGAN
22-09-20 18:53:40.487 :           GPU ID : 0
[3, 3, 64, 23, 32, 4]
22-09-20 18:53:42.684 :       Input Path : testsets/RealSRSet
22-09-20 18:53:42.684 :      Output Path : testsets/RealSRSet_results_x4
22-09-20 18:53:42.684 : ---1 --> BSRGAN --> x4--> 0000006_00611_d_0000002.jpg
22-09-20 18:53:44.913 : ---2 --> BSRGAN --> x4--> 0000063_06000_d_0000007.jpg
22-09-20 18:53:45.233 : ---3 --> BSRGAN --> x4--> 9999938_00000_d_0000295.jpg
22-09-20 18:53:45.961 : ---4 --> BSRGAN --> x4--> 9999938_00000_d_0000354.jpg
22-09-20 18:53:46.661 : ---5 --> BSRGAN --> x4--> 9999938_00000_d_0000383.jpg
22-09-20 18:53:47.367 : ---6 --> BSRGAN --> x4--> 9999952_00000_d_0000193.jpg
22-09-20 18:53:48.069 : ---7 --> BSRGAN --> x4--> 9999952_00000_d_0000287.jpg
22-09-20 18:53:48.776 : ---8 --> BSRGAN --> x4--> 9999952_00000_d_0000334.jpg
22-09-20 18:53:49.485 : ---9 --> BSRGAN --> x4--> 9999952_00000_d_0000336.jpg
22-09-20 18

In [ ]:
for filename in original_list:
  file_name = get_filename_from_file(filename)[:-4]
  old_name = '/content/gdrive/MyDrive/TFG/sreval/BSRGAN_x4/' + file_name + '_BSRGAN.png'
  new_name = '/content/gdrive/MyDrive/TFG/sreval/BSRGAN_x4/' + file_name + '.jpg'
  shutil.move(old_name, new_name)

In [ ]:
file_name = '0000006_00611_d_0000002'
old_name = '/content/BSRGAN/testsets/RealSRSet/RealSRSet_results_x2/' + file_name + '_BSRGANx2.jpg'
new_name = '/content/gdrive/MyDrive/TFG/sreval/BSRGAN_x2/' + file_name + '.jpg'

shutil.move(old_name, new_name)

In [ ]:
!rm -r /content/BSRGAN/testsets/RealSRSet/*.png

### Figures ###

#### Import results ####

In [ ]:
dict_gt = read_annotations_from_json(common.DETEVAL_DIR, 'visdrone_gt')
dict_det_base = read_annotations_from_json(common.DETEVAL_DIR, 'test300_0_5_base')
dict_det_sr = read_annotations_from_json(common.DETEVAL_DIR, 'test300_d4_0_5_sr_iou_025')
dict_det_prop = read_annotations_from_json(common.DETEVAL_DIR, 'test300_d4_0_5_srfliplut_clahe50_iou_025')

#### Bar plot average score per method ####

In [ ]:
avgscore_det_base_small = 0
avgscore_det_base_medium = 0
avgscore_det_base_large = 0

avgscore_det_sr_small = 0
avgscore_det_sr_medium = 0
avgscore_det_sr_large = 0

avgscore_det_prop_small = 0
avgscore_det_prop_medium = 0
avgscore_det_prop_large = 0

# base
for ann in dict_det_base['annotations']:
  area = ann['area']
  if area <= 32*32:
    avgscore_det_base_small += float(ann['score'])
  elif area <= 96*96:
    avgscore_det_base_medium += float(ann['score'])
  else:
    avgscore_det_base_large += float(ann['score'])

# sr
for ann in dict_det_sr['annotations']:
  area = ann['area']
  if area <= 32*32:
    avgscore_det_sr_small += float(ann['score'])
  elif area <= 96*96:
    avgscore_det_sr_medium += float(ann['score'])
  else:
    avgscore_det_sr_large += float(ann['score'])

# prop
for ann in dict_det_prop['annotations']:
  area = ann['area']
  if area <= 32*32:
    avgscore_det_prop_small += float(ann['score'])
  elif area <= 96*96:
    avgscore_det_prop_medium += float(ann['score'])
  else:
    avgscore_det_prop_large += float(ann['score'])

# get count of objects by size
small_count_base, medium_count_base, large_count_base = get_det_count_by_size(dict_det_base['annotations'])
small_count_sr, medium_count_sr, large_count_sr = get_det_count_by_size(dict_det_sr['annotations'])
small_count_prop, medium_count_prop, large_count_prop = get_det_count_by_size(dict_det_prop['annotations'])

avgscore_det_base_small /= small_count_base
avgscore_det_base_medium /= medium_count_base
avgscore_det_base_large /= large_count_base

avgscore_det_sr_small /= small_count_sr
avgscore_det_sr_medium /= medium_count_sr
avgscore_det_sr_large /= large_count_sr

avgscore_det_prop_small /= small_count_prop
avgscore_det_prop_medium /= medium_count_prop
avgscore_det_prop_large /= large_count_prop

avgscore_det_base_small = round(avgscore_det_base_small, 4)
avgscore_det_base_medium = round(avgscore_det_base_medium, 4)
avgscore_det_base_large = round(avgscore_det_base_large, 4)

avgscore_det_sr_small = round(avgscore_det_sr_small, 4)
avgscore_det_sr_medium = round(avgscore_det_sr_medium, 4)
avgscore_det_sr_large = round(avgscore_det_sr_large, 4)

avgscore_det_prop_small = round(avgscore_det_prop_small, 4)
avgscore_det_prop_medium = round(avgscore_det_prop_medium, 4)
avgscore_det_prop_large = round(avgscore_det_prop_large, 4)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

data = [[avgscore_det_base_small, avgscore_det_base_medium, avgscore_det_base_large],
        [avgscore_det_sr_small, avgscore_det_sr_medium, avgscore_det_sr_large],
        [avgscore_det_prop_small, avgscore_det_prop_medium, avgscore_det_prop_large]]

small_labels = [small_count_base, small_count_sr, small_count_prop]
medium_labels = [medium_count_base, medium_count_sr, medium_count_prop]
large_labels = [large_count_base, large_count_sr, large_count_prop]

x = np.arange(3)
fig = plt.figure()
ax = fig.add_axes([0,0,2,2])
ax.bar(x + 0.00, data[0], color = 'crimson', width = 0.25)
ax.bar(x + 0.25, data[1], color = 'gold', width = 0.25)
ax.bar(x + 0.50, data[2], color = 'royalblue', width = 0.25)

# show bar labels on top

for i, v in enumerate(data[0]):
    plt.text(x[i] - 0.07, v + 0.01, str(v), fontsize=12)

for i, v in enumerate(data[1]):
    plt.text(x[i] + 0.18, v + 0.01, str(v), fontsize=12)

for i, v in enumerate(data[2]):
    plt.text(x[i] + 0.42, v + 0.01, str(v), fontsize=12)

#plt.tick_params(axis='x', which='both', bottom=False, labelbottom=False)
plt.xticks(ticks=x + 0.25, labels = ['small', 'medium', 'large'], fontsize=16)
plt.ylim([0, 0.6])
ax.legend(labels=['BASE', 'PSR', 'PSRFLUT'], loc='upper left')
ax.set_title('Average scores by object size and method', fontsize=24)
#plt.xlabel('small medium large', fontsize=16)
ax.set_ylabel('Average score', fontsize=16)
plt.show()

#### Scatter plot detection count by image ####

In [ ]:
count_gt = np.zeros(common.TEST_MAX_SIZE).astype('int')

for ann in dict_gt['annotations']:
  count_gt[int(ann['image_id']) - 1] += 1

count_det_base = np.zeros(common.TEST_MAX_SIZE).astype('int')

for ann in dict_det_base['annotations']:
  count_det_base[int(ann['image_id']) - 1] += 1

count_det_sr = np.zeros(common.TEST_MAX_SIZE).astype('int')

for ann in dict_det_sr['annotations']:
  count_det_sr[int(ann['image_id']) - 1] += 1

count_det_prop = np.zeros(common.TEST_MAX_SIZE).astype('int')

for ann in dict_det_prop['annotations']:
  count_det_prop[int(ann['image_id']) - 1] += 1


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

x = np.arange(common.TEST_MAX_SIZE)
methods = np.array(['GT', 'BASE', 'SR', 'PROP'])

plt.figure(figsize=(24,15))
plt.scatter(x, count_gt, c='black', marker='o', label='GT')
plt.scatter(x, count_det_prop, c='royalblue', marker='^', label='PSRFLUT')
plt.scatter(x, count_det_sr, c='gold', marker='s', label='PSR')
plt.scatter(x, count_det_base, c='crimson', marker='v', label='BASE')

plt.title('Detection count by image and method (T=0.3)', fontsize=24)
plt.xlabel('Images', fontsize=16)
plt.ylabel('Detection count', fontsize=16)
plt.legend(fontsize=12)
plt.show()

#### Histogram of detections count by area size ####

In [ ]:
area_det_gt = []
area_det_base = []
area_det_sr = []
area_det_prop = []

for ann in dict_gt['annotations']:
  area_det_gt += [int(ann['area']) - 1]

for ann in dict_det_base['annotations']:
  area_det_base += [int(ann['area']) - 1]

for ann in dict_det_sr['annotations']:
  area_det_sr += [int(ann['area']) - 1]

for ann in dict_det_prop['annotations']:
  area_det_prop += [int(ann['area']) - 1]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

bin_width = 128
n_bins = 100
bins = [bin_width*x for x in np.arange(n_bins)]
hist, _ = np.histogram(area_det_base, bins)

plt.figure(figsize=(24,15))
plt.xticks(np.arange(0, np.max(bins), step=n_bins*10))
plt.hist(area_det_gt, bins, color='black')
plt.hist(area_det_prop, bins, color='royalblue')
plt.hist(area_det_sr, bins, color='gold')
plt.hist(area_det_base, bins, color='crimson')
plt.legend(labels=['GT', 'PSRFLUT', 'PSR','BASE'])
plt.axvline(x=1024, color='dimgrey', linestyle='--', linewidth=2)
plt.axvline(x=9216, color='dimgrey', linestyle='--', linewidth=2)
plt.title('Histogram of detections by object area and method (T=0.5)', fontsize=24)
plt.xlabel('Área del objeto', fontsize=16)
plt.ylabel('Número de detecciones', fontsize=16)
plt.show()

#### Count detections by object size group ####

In [ ]:
small_gt, medium_gt, large_gt = get_det_count_by_size(dict_gt['annotations'])
small_base, medium_base, large_base = get_det_count_by_size(dict_det_base['annotations'])
small_sr, medium_sr, large_sr = get_det_count_by_size(dict_det_sr['annotations'])
small_prop, medium_prop, large_prop = get_det_count_by_size(dict_det_prop['annotations'])

print('GT:', small_gt, medium_gt, large_gt)
print('BASE:', small_base, medium_base, large_base)
print('SRx2:', small_sr, medium_sr, large_sr)
print('PROP:', small_prop, medium_prop, large_prop)

# ratio by method
ratio_base = 100*round(small_base/small_gt, 4)
print("ratio small base/gt: ", ratio_base)
ratio_sr = 100*round(small_sr/small_gt, 4)
print("ratio small sr/gt: ", ratio_sr)
ratio_prop = 100*round(small_prop/small_gt, 4)
print("ratio small prop/gt: ", ratio_prop)

#### Image Tiling ####

In [ ]:
def get_image_by_filename(filename):
  image_file = os.path.join(IMAGES_DIR, filename) + '.jpg'
  return get_image_from_file(image_file, as_rgb=False, as_tensor=False, as_type='uint8')

img = get_image_by_filename('9999979_00000_d_0000045')
width, height = get_image_shape(img)

x_tiles, y_tiles = get_image_tiling(width,height)
tiles_dict = get_tiles(width, height, x_tiles, y_tiles)

"""
for tile in tiles_dict['tiles']:
  img = cv.rectangle(img, (tile['ymin'],tile['xmin']), (tile['ymax'],tile['xmax']), (0, 255, 255), 2)

tile = tiles_dict['tiles'][3]
img = cv.rectangle(img, (tile['ymin'],tile['xmin']), (tile['ymax'],tile['xmax']), (0, 0, 255), 2)

tile = tiles_dict['tiles'][3]
crop_img = img[tile['xmin']:tile['xmax'],tile['ymin']:tile['ymax']]
print(crop_img.shape)
cv2_imshow(crop_img)

path = BASE_DIR + '/figures/'
file_name = 'figure3c.png'
write_image_to_file(path, file_name, img)
"""

#### LUT images and plot ####

In [ ]:
#images with low brightness
images_list = [
    '/content/gdrive/MyDrive/TFG/VisDrone_2019-DET-test-dev/images/9999952_00000_d_0000085.jpg',
    '/content/gdrive/MyDrive/TFG/VisDrone_2019-DET-test-dev/images/9999952_00000_d_0000164.jpg',
    '/content/gdrive/MyDrive/TFG/VisDrone_2019-DET-test-dev/images/9999952_00000_d_0000283.jpg',
    '/content/gdrive/MyDrive/TFG/VisDrone_2019-DET-test-dev/images/9999963_00000_d_0000073.jpg',
    '/content/gdrive/MyDrive/TFG/VisDrone_2019-DET-test-dev/images/9999980_00000_d_0000004.jpg'
]

In [ ]:
def get_clahe_lut_from_images(image_file_in, image_file_out):
  image_in = get_image_from_file(image_file_in, True, False, 'uint8').flatten()
  image_out = get_image_from_file(image_file_out, True, False, 'uint8').flatten()

  N = len(image_in)
  map = {}

  for i in range(N):
    k = str(image_in[i])
    v = str(image_out[i])
    map[k] = v

  x = np.arange(256)
  y = []

  list_keys = map.keys()
  for k in x:
    if str(k) in list_keys:
      y += [int(map[str(k)])]
    else:
      y += [k]

  return y

In [ ]:
image_file = common.TEST_DIR + '9999952_00000_d_0000145.jpg'
image_file_out = common.TEST_DIR + '9999952_00000_d_0000145_clahe.jpg'
image = get_image_from_file(image_file, True, False, 'uint8')

gamma_lut = get_gamma_lut(image)
increase_lut = get_increase_lut(image)
clahe_lut = get_clahe_lut_out_from_images(image_file, image_file_out)

In [ ]:
fig = plt.figure()
fig.set_figheight(20)
fig.set_figwidth(20)

image_file_out = common.TEST_DIR + '9999952_00000_d_0000145_increase.jpg'
image_out = get_image_from_file(image_file_out, True, False, 'uint8')

plt.axis('off')
plt.title('Imagen aumentada con LUT', fontsize=18)
plt.imshow(image_out)

plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure()
fig.set_figheight(8)
fig.set_figwidth(8)

x = np.arange(256)
plt.plot(x, clahe_lut, c='red', linewidth=2)
plt.title('CLAHE LUT', fontsize=16)
plt.xlabel('IN Values', fontsize=12)
plt.ylabel('OUT Values', fontsize=12)
plt.show()